# aifood baseline 

### 本baseline采用pytorch框架，应用ModelArts的Notebook进行开发

### 数据集获取
将您OBS桶中的数据文件加载到此notebook中，将如下代码中"obs-aifood-baseline"修改成您OBS桶名称。

In [1]:
import moxing as mox
mox.file.copy_parallel('s3://ai-awe-n4/aifood','./aifood/')
print("done")

INFO:root:Using MoXing-v1.14.0-14d5d81b
INFO:root:Using OBS-Python-SDK-3.1.2
INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:pid: None.	1000/5001
INFO:root:pid: None.	2000/5001
INFO:root:pid: None.	3000/5001
INFO:root:pid: None.	4000/5001
INFO:root:pid: None.	5000/5001


done


### 加载依赖

In [2]:

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import time
import os


### 加载数据集，并将其分为训练集和测试集

In [6]:
dataTrans = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
#     transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
 
    # image data path
data_dir = './aifood/images'
all_image_datasets = datasets.ImageFolder(data_dir, dataTrans)
#print(all_image_datasets.class_to_idx)    
trainsize = int(0.8*len(all_image_datasets))
testsize = len(all_image_datasets) - trainsize
train_dataset, test_dataset = torch.utils.data.random_split(all_image_datasets,[trainsize,testsize])
   
image_datasets = {'train':train_dataset,'val':test_dataset}
    

    # wrap your data and label into Tensor

    
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                 batch_size=64,
                                                 shuffle=True,
                                                 num_workers=4) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

    # use gpu or not
use_gpu = torch.cuda.is_available()

In [4]:
def train_model(model, lossfunc, optimizer, scheduler, num_epochs=10):
    start_time = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for data in dataloders[phase]:
                # get the inputs
                inputs, labels = data
                

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = lossfunc(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data).to(torch.float32)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    elapsed_time = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(
        elapsed_time // 60, elapsed_time % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
  
    return model

In [13]:
# get model and replace the original fc layer with your fc layer
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 10)

if use_gpu:
    model_ft = model_ft.cuda()

    # define loss function
lossfunc = nn.CrossEntropyLoss()

    # setting optimizer and trainable parameters
 #   params = model_ft.parameters()
 # list(model_ft.fc.parameters())+list(model_ft.layer4.parameters())
#params = list(model_ft.fc.parameters())+list( model_ft.parameters())
params = list(model_ft.fc.parameters())
optimizer_ft = optim.SGD(params, lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)

model_ft = train_model(model=model_ft,
                       lossfunc=lossfunc,
                       optimizer=optimizer_ft,
                       scheduler=exp_lr_scheduler,
                       num_epochs=30)

Epoch 0/29
----------


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ma-user/anaconda3/envs/Pytorch-1.0.0/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/ma-user/anaconda3/envs/Pytorch-1.0.0/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ma-user/anaconda3/envs/Pytorch-1.0.0/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ma-user/anaconda3/envs/Pytorch-1.0.0/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/ma-user/anaconda3/envs/Pytorch-1.0.0/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/ma-user/anaconda3/envs/Pytorch-1.0.0/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
   

In [10]:
torch.save(model_ft.state_dict(), './model.pth')

### 模型训练
采用resnet50神经网络结构训练模型,模型训练需要一定时间，等待该段代码运行完成后再往下执行。

In [8]:
# 构建多模型融合结构
model_1 = models.vgg16(pretrained = True)
model_2 = models.resnet50(pretrained = True)

# 设置模型的参数不需要进行梯度下降
for param in model_1.parameters():
    param.requires_grad = False
model_1.classifier = torch.nn.Sequential(torch.nn.Linear(25088,4096),
                                        torch.nn.ReLU(),
                                        torch.nn.Dropout(p = 0.5),
                                        torch.nn.Linear(4096,4096),
                                        torch.nn.ReLU(),
                                        torch.nn.Dropout(p = 0.5),
                                        torch.nn.Linear(4096,10))
for param in model_2.parameters():
    param.requires_grad = False
num_ftrs = model_2.fc.in_features
model_2.fc = torch.nn.Linear(num_ftrs,10)

Use_gpu = torch.cuda.is_available()
# 设置损失函数以及优化方法
if Use_gpu:
    model_1 = model_1.cuda()
    model_2 = model_2.cuda()
    
loss_f_1 = torch.nn.CrossEntropyLoss()
loss_f_2 = torch.nn.CrossEntropyLoss()

optimizer_1 = torch.optim.Adam(model_1.classifier.parameters(), lr = 0.00001)
params_2 = list(model_ft.fc.parameters())
optimizer_2 = optim.SGD(params_2, lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_2, step_size=7, gamma=0.1)

# 设置两个模型融合的权重参数
weight_1 = 0.6
weight_2 = 0.4

In [18]:
def train_model_cross(model_1,model_2, loss_f_1,loss_f_2, optimizer_1,optimizer_2,weight_1,weight_2, scheduler, num_epochs=10):
    start_time = time.time()

    best_model_wts_1 = model_1.state_dict()
    best_model_wts_2 = model_2.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model_1.train(True)
                model_2.train(True)  # Set model to training mode
            else:
                model_1.train(False)
                model_2.train(False)  # Set model to evaluate mode

            running_loss_1 = 0.0
            running_loss_2 = 0.0
            running_corrects_1 = 0.0
            running_corrects_2 = 0.0
            blending_running_corrects = 0.0

            # Iterate over data.
            for data in dataloders[phase]:
                # get the inputs
                inputs, labels = data
                

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)


                # forward
                y_pred_1 = model_1(inputs)
                y_pred_2 = model_2(inputs)
                blending_y_pred = y_pred_1 * weight_1 + y_pred_2 * weight_2
                _, pred_1 = torch.max(y_pred_1.data,1) # 找出每一行最大值对应的索引值
                _, pred_2 = torch.max(y_pred_2.data,1)
                _, blending_y_pred = torch.max(blending_y_pred.data,1)
                
                optimizer_1.zero_grad()
                optimizer_2.zero_grad()

                loss_1 = loss_f_1(y_pred_1,labels)
                loss_2 = loss_f_2(y_pred_2,labels)
                

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss_1.backward()
                    loss_2.backward()
                    optimizer_1.step()
                    optimizer_2.step()
                    
                # statistics
                running_loss_1 += loss_1.data
                running_loss_2 += loss_2.data
                running_corrects_1 += torch.sum(pred_1 == labels.data).to(torch.float32)
                running_corrects_2 += torch.sum(pred_2 == labels.data).to(torch.float32)
                blending_running_corrects += torch.sum(blending_y_pred == labels.data).to(torch.float32)

            
            epoch_loss_1 = running_loss_1/dataset_sizes[phase]
            epoch_acc_1 = running_corrects_1 * 100/dataset_sizes[phase]
            epoch_loss_2 = running_loss_2/dataset_sizes[phase]
            epoch_acc_2 = running_corrects_2 * 100/dataset_sizes[phase]
            epoch_blending_acc = blending_running_corrects * 100/dataset_sizes[phase]

            print('{}, Model1 Loss:{:.4f},Model1 ACC:{:.4f}%,Model2 Loss:{:.4f},Model2 ACC:{:.4f}%,Blending_Model ACC:{:.4f}'
                  .format(phase,epoch_loss_1,epoch_acc_1,epoch_loss_2,epoch_acc_2,epoch_blending_acc))

            # deep copy the model
            if phase == 'val' and epoch_blending_acc > best_acc:
                best_acc = epoch_blending_acc
                best_model_wts_1 = model_1.state_dict()
                best_model_wts_2 = model_2.state_dict()

    elapsed_time = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(
        elapsed_time // 60, elapsed_time % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model_1.load_state_dict(best_model_wts_1)
    model_2.load_state_dict(best_model_wts_2)
  
    return model_1,model_2

In [19]:
model_1,model_2 = train_model_cross(
    model_1,model_2, loss_f_1,loss_f_2, optimizer_1,optimizer_2,weight_1,weight_2, exp_lr_scheduler, num_epochs=30)

Epoch 0/29
----------
train, Model1 Loss:0.0055,Model1 ACC:89.9250%,Model2 Loss:0.0373,Model2 ACC:10.4250%,Blending_Model ACC:89.5000
val, Model1 Loss:0.0065,Model1 ACC:87.6000%,Model2 Loss:0.0378,Model2 ACC:10.3000%,Blending_Model ACC:86.8000
Epoch 1/29
----------
train, Model1 Loss:0.0045,Model1 ACC:92.3250%,Model2 Loss:0.0373,Model2 ACC:10.5500%,Blending_Model ACC:91.8750
val, Model1 Loss:0.0062,Model1 ACC:87.0000%,Model2 Loss:0.0378,Model2 ACC:10.2000%,Blending_Model ACC:87.2000
Epoch 2/29
----------
train, Model1 Loss:0.0037,Model1 ACC:93.7750%,Model2 Loss:0.0373,Model2 ACC:10.5750%,Blending_Model ACC:93.6500
val, Model1 Loss:0.0056,Model1 ACC:89.5000%,Model2 Loss:0.0378,Model2 ACC:9.9000%,Blending_Model ACC:88.8000
Epoch 3/29
----------
train, Model1 Loss:0.0032,Model1 ACC:94.6750%,Model2 Loss:0.0373,Model2 ACC:10.3250%,Blending_Model ACC:94.7500
val, Model1 Loss:0.0055,Model1 ACC:89.2000%,Model2 Loss:0.0377,Model2 ACC:10.3000%,Blending_Model ACC:88.6000
Epoch 4/29
----------
tra

将训练好的模型保存下来。

In [20]:
# torch.save(model_ft.state_dict(), './model.pth')
torch.save({
            'model_1_state_dict': model_1.state_dict(),
            'model_2_state_dict': model_2.state_dict()
            }, './model_cross.pth')

In [22]:
model_1 = models.vgg16(pretrained = True)
model_2 = models.resnet50(pretrained = True)

for param in model_1.parameters():
    param.requires_grad = False
model_1.classifier = torch.nn.Sequential(torch.nn.Linear(25088,4096),
                                        torch.nn.ReLU(),
                                        torch.nn.Dropout(p = 0.5),
                                        torch.nn.Linear(4096,4096),
                                        torch.nn.ReLU(),
                                        torch.nn.Dropout(p = 0.5),
                                        torch.nn.Linear(4096,10))
for param in model_2.parameters():
    param.requires_grad = False
num_ftrs = model_2.fc.in_features
model_2.fc = torch.nn.Linear(num_ftrs,10)

load_name = './model_cross.pth'
checkpoint = torch.load(load_name,map_location ='cpu')
model_1.load_state_dict(checkpoint['model_1_state_dict'])
model_2.load_state_dict(checkpoint['model_2_state_dict'])
model_1.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

### 将训练好的模型保存至OBS
将模型保存到OBS桶中model文件夹下，为后续推理测试、模型提交做准备。将如下代码中"obs-aifood-baseline"修改成您OBS桶的名称。


In [11]:
import moxing as mox
mox.file.copy('./model.pth','s3://ai-awe-n4/model_output/model/resnet-50-torch.pth')
print("done")

done


In [23]:
import moxing as mox
mox.file.copy('./model_cross.pth','s3://ai-awe-n4/model_output1/model/vgg-resnet-50-torch.pth')
print("done")

done
